# Fine-tuning Stable Diffusion v1-4 con Old Book Illustrations

Adaptacion del notebook (`2.finetuning_stable_diffusion.ipynb`)
para el dataset `gigant/oldbookillustrations`.

**3 cambios:**
1. `example["image"]` -> `example["1600px"]` (columna de imagen)
2. `example["text"]` -> `example["info_alt"]` (columna de caption)
3. `Resize((512,512))` -> `Resize(256) + CenterCrop(256)` (imagenes no cuadradas)

**Config actual:** 1000 muestras, 256px, batch_size=4

In [ ]:
# Importamos las librerias necesarias:
from diffusers import StableDiffusionPipeline, DDPMScheduler
from diffusers import UNet2DConditionModel, AutoencoderKL
from transformers import CLIPTextModel, CLIPTokenizer
from datasets import load_dataset
from accelerate import Accelerator
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image, ImageFile
import os
from tqdm import tqdm
import time
from datetime import datetime, timedelta

# Permitir imagenes truncadas del dataset
ImageFile.LOAD_TRUNCATED_IMAGES = True

def format_eta(seconds):
    """Formatea segundos a string legible."""
    if seconds < 0:
        return "N/A"
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    if h > 0:
        return f"{h}h {m:02d}m"
    return f"{m}m {int(seconds % 60):02d}s"

In [ ]:
# Vamos a utilizar GPU si esta disponible, o CPU:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
pretrained_model_name = "CompVis/stable-diffusion-v1-4"

# Cargamos un modelo pre-entrenado Stable Diffusion:
pipe = StableDiffusionPipeline.from_pretrained(
    pretrained_model_name,
).to(device)

In [ ]:
# Generamos una imagen ANTES del fine-tuning:
prompt = "an illustration of a ship sailing through a stormy sea"
image_before = pipe(prompt, height=256, width=256).images[0]

image_before.save("../generated/before_finetuning.png")
image_before

In [ ]:
# Cargando el dataset Old Book Illustrations:
dataset_name = "gigant/oldbookillustrations"
max_train_samples = 1000  # Limitar muestras para CPU

dataset = load_dataset(dataset_name, split="train")
if max_train_samples:
    dataset = dataset.select(range(max_train_samples))

# Comprobamos el tamano de las imagenes del dataset:
size = dataset[0]["1600px"].size
print(f"Tamano de las imagenes del dataset: {size}")
print(f"Numero de muestras: {len(dataset)}")

In [ ]:
# Definimos las transformaciones necesarias para el dataset:
# Resize(256) + CenterCrop(256) en vez de Resize((512,512))
# Las imagenes de Old Book Illustrations no son cuadradas,
# asi que usamos Resize + CenterCrop para preservar proporciones.
resolution = 256
image_transforms = transforms.Compose([
    transforms.Resize(resolution),                          # redimensionar lado menor a 256
    transforms.CenterCrop(resolution),                      # recortar cuadrado central
    transforms.ToTensor(),                                   # convertir a tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),  # normalizacion
])

In [ ]:
# Aplicamos la transformacion a una imagen aleatoria del dataset:
original_image = dataset[0]["1600px"]
transformed_image = image_transforms(original_image.convert("RGB"))
transformed_pil_image = transforms.ToPILImage()(transformed_image)

print("Imagen transformada:")
transformed_pil_image

In [ ]:
# Antes de cargar todas las componentes, liberamos la pipeline:
del pipe
if device == "cuda":
    torch.cuda.empty_cache()

In [ ]:
# Tokenizador:
tokenizer = CLIPTokenizer.from_pretrained(pretrained_model_name, subfolder='tokenizer')

# Scheduler:
noise_scheduler = DDPMScheduler.from_pretrained(pretrained_model_name, subfolder="scheduler")

# Text Encoder (CLIP):
text_encoder = CLIPTextModel.from_pretrained(
    pretrained_model_name,
    subfolder="text_encoder",
).to(device)

# VAE: Autoencoder:
vae = AutoencoderKL.from_pretrained(
    pretrained_model_name,
    subfolder="vae",
).to(device)

# La UNet:
unet = UNet2DConditionModel.from_pretrained(
    pretrained_model_name,
    subfolder="unet",
).to(device)

In [ ]:
# Creamos un Dataset wrapper para la hora del entrenamiento:
batch_size = 4

class Text2ImageDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        example = self.dataset[idx]
        # CAMBIO 1: "1600px" en vez de "image"
        image = image_transforms(example["1600px"].convert("RGB"))
        # CAMBIO 2: "info_alt" en vez de "text"
        token = tokenizer(example["info_alt"], padding="max_length", truncation=True, max_length=tokenizer.model_max_length, return_tensors="pt")
        return {
            "pixel_values": image,
            "input_ids": token.input_ids.squeeze(0),
            "attention_mask": token.attention_mask.squeeze(0),
        }

train_dataset = Text2ImageDataset(dataset)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Congelamos los pesos del VAE y del Text Encoder, ya que solo queremos finetunear la UNet:
vae.eval()
text_encoder.eval()

for param in vae.parameters():
    param.requires_grad = False
for param in text_encoder.parameters():
    param.requires_grad = False

In [ ]:
# Optimizador:
learning_rate = 1e-5
optimizer = torch.optim.AdamW(unet.parameters(), lr=learning_rate)

# Acelerador:
accelerator = Accelerator()
unet, optimizer, train_dataloader = accelerator.prepare(unet, optimizer, train_dataloader)
print(accelerator.device)

In [ ]:
# Training loop:
num_epochs = 2

os.makedirs("../outputs/checkpoints", exist_ok=True)

batches_per_epoch = len(train_dataloader)
total_batches = num_epochs * batches_per_epoch
print(f"Batches por epoch: {batches_per_epoch}, total: {total_batches}")

global_step = 0
start_time = time.time()
last_checkpoint_pct = 0

for epoch in range(num_epochs):
    epoch_start = time.time()
    epoch_losses = []
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}")

    for batch in progress_bar:
        batch_start = time.time()

        # Se pasan los pixeles al espacio latente con el encoder del VAE:
        with torch.no_grad():
            latents = vae.encode(batch["pixel_values"].to(accelerator.device)).latent_dist.sample()
            latents = latents * 0.18215

        # Proceso de difusion hacia delante:
        noise = torch.randn_like(latents)
        timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (latents.shape[0],), device=latents.device).long()
        noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

        # Codificamos el texto:
        encoder_hidden_states = text_encoder(batch["input_ids"].to(accelerator.device))[0]

        # Prediccion de ruido:
        noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample

        # Loss y backpropagation:
        loss = torch.nn.functional.mse_loss(noise_pred, noise)
        accelerator.backward(loss)
        optimizer.step()
        optimizer.zero_grad()

        epoch_losses.append(loss.item())
        global_step += 1
        batch_time = time.time() - batch_start

        # Progreso global
        pct = global_step / total_batches * 100
        elapsed = time.time() - start_time
        avg_time_per_batch = elapsed / global_step
        eta_seconds = avg_time_per_batch * (total_batches - global_step)

        progress_bar.set_postfix(
            loss=f"{loss.item():.4f}",
            pct=f"{pct:.0f}%",
            eta=format_eta(eta_seconds),
        )

        # Log detallado cada 10 batches
        if global_step % 10 == 0 or global_step == 1:
            print(f"[Batch {global_step}/{total_batches}] "
                  f"[Global {pct:.0f}%] "
                  f"Loss: {loss.item():.4f} | "
                  f"{batch_time:.1f}s/batch | "
                  f"ETA: {format_eta(eta_seconds)}")

        # Checkpoint cada 10% de progreso
        current_pct_bucket = int(pct // 10) * 10
        if current_pct_bucket > last_checkpoint_pct and current_pct_bucket > 0:
            last_checkpoint_pct = current_pct_bucket
            ckpt_path = f"../outputs/checkpoints/checkpoint-pct-{current_pct_bucket}"
            unet.save_pretrained(ckpt_path)
            print(f"--- Checkpoint {current_pct_bucket}% guardado en {ckpt_path} ---")

    # Fin de epoch
    epoch_time = time.time() - epoch_start
    avg_loss = sum(epoch_losses) / len(epoch_losses)
    print(f"Epoch {epoch + 1}/{num_epochs} completado - "
          f"Loss promedio: {avg_loss:.6f} - "
          f"Tiempo: {format_eta(epoch_time)}")

    # Checkpoint cada epoch
    checkpoint_path = f"../outputs/checkpoints/checkpoint-epoch-{epoch + 1}"
    unet.save_pretrained(checkpoint_path)
    print(f"Checkpoint epoch guardado en {checkpoint_path}")

total_time = time.time() - start_time
print(f"\nEntrenamiento completado en {format_eta(total_time)}")

In [ ]:
# Guardamos el modelo finetuneado:
output_dir = "../outputs/finetuned-model"
os.makedirs(output_dir, exist_ok=True)

unet.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Modelo guardado en {output_dir}")

## Inferencia con el modelo fine-tuneado

Cargamos la UNet fine-tuneada y generamos una imagen con el mismo prompt.

In [ ]:
# Cargamos la UNet finetuneada:
finetuned_unet = UNet2DConditionModel.from_pretrained("../outputs/finetuned-model")
finetuned_unet.to(device)

print('Modelo finetuneado cargado correctamente!')

In [ ]:
from diffusers import StableDiffusionPipeline

# Cargamos el modelo pre-entrenado pero sustituyendo la UNet por la nuestra:
pipe = StableDiffusionPipeline.from_pretrained(
    pretrained_model_name,
    unet=finetuned_unet,
).to(device)

In [ ]:
# Generamos una imagen con el modelo finetuneado (mismo prompt que antes):
prompt = "an illustration of a ship sailing through a stormy sea"
image_after = pipe(prompt, height=256, width=256).images[0]

image_after.save("../generated/after_finetuning.png")
image_after

In [ ]:
# Comparacion side-by-side
import matplotlib.pyplot as plt

image_before = Image.open("../generated/before_finetuning.png")

fig, axes = plt.subplots(1, 2, figsize=(14, 7))

axes[0].imshow(image_before)
axes[0].set_title('ANTES del fine-tuning', fontsize=14)
axes[0].axis('off')

axes[1].imshow(image_after)
axes[1].set_title('DESPUES del fine-tuning', fontsize=14)
axes[1].axis('off')

plt.suptitle(f'Prompt: "{prompt}"', fontsize=12, style='italic')
plt.tight_layout()
plt.savefig("../generated/comparison.png", dpi=150, bbox_inches='tight')
plt.show()

print("Comparacion guardada en ../generated/comparison.png")